# Loan Prediction

## 1. Introduction

### Problem Definition

We are in front a Binary Classification problem, as the aim of this notebook is to predict which of the costumers will have their loan approved.

### The Data

This dataset was obtained from Kaggle and is named Loan Prediction Dataset. It contains a set of 613 records under 13 attributes:

* **Loan_ID:** A uniques loan ID
* **Gender:** Male/Female
* **Married:** Married(Yes)/Not married(No)  
* **Dependents:** Number of persons depending on the client.
* **Education:** Application Education (Graduate / Undergraduate)
* **Self_Employed:** Self employed (Yes/No)
* **ApplicantIncome:** Applicant income
* **CoapplicantIncome:** Coapplicant Income
* **LoanAmount:** Loan amount in thousands
* **Loan_Amount_Term:** Term of lean in months
* **Credit_History:** Credit history meets guidelines
* **Property_Area:** Urban/Semi and Rural
* **Loan_Status:** Loan approved (Y,N)

As we said, the main objective of this probles is to use machine learning techniques to predict loan payments, therefore, our target value is Loan_Status.

### Preparing the tools

In [4]:
# Basic libraries
import sys
import sklearn
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(64)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

print("Set up completed")

ModuleNotFoundError: No module named 'sklearn'

### Load the Data